loading all the packages

In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

In [7]:
# Load the reference object image (silhouette)
reference_image_path = 'reference.png'
reference_image = cv2.imread(reference_image_path, cv2.IMREAD_UNCHANGED)

In [8]:
# Check if the reference image is loaded correctly
if reference_image is None:
    raise FileNotFoundError(f"Reference image not found at {reference_image_path}")

In [9]:
# Add an alpha channel to the reference image if it doesn't have one
if reference_image.shape[2] == 3:
    b, g, r = cv2.split(reference_image)
    alpha = np.ones(b.shape, dtype=b.dtype) * 255  # Create a white alpha channel
    reference_image = cv2.merge((b, g, r, alpha))

In [10]:
# Make the reference image translucent
reference_image[:, :, 3] = 100

In [11]:
# Define a function to calculate the Euclidean distance between two points
def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

In [19]:
# Function to estimate body measurements in centimeters
def estimate_body_measurements(landmarks, conversion_factor):
    measurements = {}
    
    chest_circumference = calculate_distance(
        (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y),
        (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
    ) * 2 * conversion_factor[1]
    
    waist_circumference = calculate_distance(
        (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y),
        (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)
    ) * 2 * conversion_factor[2]
    
    hip_circumference = calculate_distance(
        (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y),
        (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)
    ) * 2 * conversion_factor[3]

    inseam_length = calculate_distance(
        (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y),
        (landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y)
    ) * conversion_factor[4]
    
    # Calculate height from ankle to shoulder as a simple approximation
    height = (calculate_distance(
        (landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y),
        (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
    ) + calculate_distance(
        (landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y),
        (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
    )) / 2 * conversion_factor[7]

    measurements['Height'] = height
    measurements['Chest Circumference'] = chest_circumference
    measurements['Waist Circumference'] = waist_circumference
    measurements['Hip Circumference'] = hip_circumference
    measurements['Inseam Length'] = inseam_length

    return measurements

In [14]:
# Function to overlay the reference image
def overlay_image_alpha(img, img_overlay, pos, alpha_mask=None):
    x, y = pos

    # Image ranges
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    # Overlay the image
    img_crop = img[y1:y2, x1:x2]
    img_overlay_crop = img_overlay[y1o:y2o, x1o:x2o]

    if alpha_mask is not None:
        alpha = alpha_mask[y1o:y2o, x1o:x2o, np.newaxis] / 255.0
        img_crop[:] = alpha * img_overlay_crop + (1.0 - alpha) * img_crop
    else:
        img_crop[:] = img_overlay_crop

In [15]:
def is_visible_from_head_to_toe(landmarks):
    required_landmarks = [
        mp_pose.PoseLandmark.LEFT_EYE,
        mp_pose.PoseLandmark.RIGHT_EYE,
        mp_pose.PoseLandmark.LEFT_FOOT_INDEX,
        mp_pose.PoseLandmark.RIGHT_FOOT_INDEX
    ]
    return all(landmarks[landmark.value].visibility > 0.5 for landmark in required_landmarks)


In [22]:
# Dictionary to store all measured values as lists
all_measurements = {
    'Height': [],
    'Chest Circumference': [],
    'Waist Circumference': [],
    'Hip Circumference': [],
    'Inseam Length': [],
}

In [31]:
# Start capturing video from the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform pose detection
    results = pose.process(image_rgb)

    # Resize the reference image to match the frame size
    ref_height, ref_width, _ = image.shape
    resized_ref_image = cv2.resize(reference_image, (ref_width, ref_height))

    # Overlay the reference image on the frame
    overlay_image_alpha(image, resized_ref_image[:, :, :3], (0, 0), resized_ref_image[:, :, 3])

    # Draw the pose annotation on the image
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract the landmarks
        landmarks = results.pose_landmarks.landmark

        # Check if the person is visible from head to toe
        if is_visible_from_head_to_toe(landmarks):
            # Estimate body measurements (example conversion factor, adjust as needed)
            conversion_factor = [254,283,422,520,175,180,618,210]  # Placeholder conversion factor
            measurements = estimate_body_measurements(landmarks, conversion_factor)
            
            # Store the measurements in the dictionary as lists
            for key, value in measurements.items():
                all_measurements[key].append(value)
            
            # Display the measurements on the image
            y_offset = 30
            for key, value in measurements.items():
                cv2.putText(image, f'{key}: {value:.2f} cm', (10, y_offset), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                y_offset += 20

            
    # Display the image
    cv2.imshow('Pose Estimation with Reference Overlay', image)

    # Break the loop on 'q' key press
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

c:\Users\reddy\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [32]:
for key, values in all_measurements.items():
    print(f'{key}: {round(sum(values)/len(values))}')

Height: 159
Chest Circumference: 90
Waist Circumference: 80
Hip Circumference: 99
Inseam Length: 79


In [34]:
def classify_size(bust, waist, hips):
    size_chart = {
        'XS': {'bust': (79, 84), 'waist': (60, 65), 'hips': (84, 89)},
        'S': {'bust': (84, 89), 'waist': (65, 70), 'hips': (89, 94)},
        'M': {'bust': (89, 94), 'waist': (70, 75), 'hips': (94, 99)},
        'L': {'bust': (94, 99), 'waist': (75, 80), 'hips': (99, 104)},
        'XL': {'bust': (99, 104), 'waist': (80, 85), 'hips': (104, 109)},
        'XXL': {'bust': (104, 109), 'waist': (85, 90), 'hips': (109, 114)},
        'XXXL': {'bust': (109, 114), 'waist': (90, 95), 'hips': (114, 119)},
    }

    possible_sizes = []

    for size, measurements in size_chart.items():
        if measurements['bust'][0] <= bust <= measurements['bust'][1] or \
           measurements['waist'][0] <= waist <= measurements['waist'][1] or \
           measurements['hips'][0] <= hips <= measurements['hips'][1]:
            possible_sizes.append(size)

    if possible_sizes:
        return possible_sizes
    else:
        return ["Size not found"]

# Example usage
chest = round(sum(all_measurements['Chest Circumference'])/len(all_measurements['Chest Circumference']))
waist = round(sum(all_measurements['Waist Circumference'])/len(all_measurements['Waist Circumference']))
hips = round(sum(all_measurements['Hip Circumference'])/len(all_measurements['Hip Circumference']))

sizes = classify_size(chest, waist, hips)
print(f"SUGGESTED SIZES: {sizes}")


SUGGESTED SIZES: ['M', 'L', 'XL']
